In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [2]:
url_base='http://www.csmpt.org.tw'
url_work_table='/news/index.php?type=4'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [5]:
pages_link=[]
for o in soup.find(class_="page").find_all('span'):
    if o.find('a'):
        O=o.find("a")
        if O.text == "到最後>|" or O.text == ">":
            pass
        else:
            link = O.get('href')
            pages_link.append(link)
            print(O.text,link)

2 ?sub=2&continue=Y&type=4
3 ?sub=3&continue=Y&type=4


# Get work table

The `get_work_page` function is used for getting each work page

In [6]:
work_table = soup.find_all('article')[0]

In [7]:
def get_work_page(page_url,page=None):
    global url
    if page != None:
        page_url = page_url+'?page='+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

Using `get_work_page` to read each page and using `get_tables_part` read work table

In [8]:
def get_work_table(table_,work_table):
    for i, item in enumerate(table_):
        if item.find('a'):
            s=item.find('a')
            if s == -1:
                s = item
                print(i, s)
            else:
                title = s.get('title')
                link = s.get('href')
                link_s = urlparse(url)._replace(path=link,query=[]).geturl()
                all_td = item.find_all('td')
                origination = all_td[1].find('p').text
                dead_line = all_td[-1].find('p').text
                print(i,title,link_s,origination,dead_line)
                work_table.append([i-2, title, dead_line, link_s ])
    return work_table

In [9]:
work_table=[]
if type(pages_link) != list:
    table_ = work_tables[0]
    work_table = get_work_table(table_,work_table)
else:
    for p_i, p_item in enumerate(pages_link):
        soup_ = get_work_page(p_item)
        table_ = get_tables_part(soup_)[0]
        
        work_table = get_work_table(table_,work_table)


MissingSchema: Invalid URL '?sub=2&continue=Y&type=4': No scheme supplied. Perhaps you meant http://?sub=2&continue=Y&type=4?

In [34]:
work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','期限' ,'連結'])

print(work_table)

    no                               召聘職稱         期限  \
0   -1                        胸腔內科-短期研究助理  111-05-22   
1    1                 急診醫學科-111年度第一年住院醫師  111-05-31   
2    3                    心臟內科研究助理(非院內計畫)  111-06-30   
3    5                       放射線部-契約醫事放射師  111-05-05   
4    7                麻醉部招訓第七期麻醉護理人員訓練班公告  111-05-13   
5    9                  大武分院徵契約物理治療師(PGY)  111-05-02   
6   11                口腔醫學部招訓111年度第一年住院醫師  111-05-04   
7   13                      品質管理中心徵契約行政助理  111-05-04   
8   15      病理檢驗部(屏東大武院區)契約醫事檢驗師(儲備) 甄選公告  111-05-06   
9   17                  耳鼻喉頭頸部契約語言治療師甄選公告  111-05-08   
10  19            護理部 大武院區儲備契約護理(手術專責護理師)  111-05-26   
11  21  心臟血管外科(屏東大武院區)-契約醫事技術師(體外循環師）甄選公告  111-06-30   
12  23                  大武院區儲備契約護理(專科護理師)  111-05-26   
13  25                        重症醫學部聘用住院醫師  111-05-15   
14  27               重建整形外科(屏東大武院區)契約主治醫師  111-05-31   
15  29              護理部大武院區及本院血液透析室儲備契約護理  111-05-06   
16  31                     大武院區及本院儲備契約護理。  111-0